<a href="https://colab.research.google.com/github/mcdulltii/CSC2102/blob/llm/llm/llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install -q --upgrade pip
!pip install -q tensorflow transformers[sentencepiece] accelerate sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 13.0 MB/s eta 0:00:00


In [ ]:
!nvidia-smi

Mon Jan 22 13:55:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import tensorflow as tf
assert len(tf.config.list_physical_devices('GPU')) > 0

In [ ]:
import time
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# LLMs

In [ ]:
prompt = """
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
%s

### Response:
"""

## BioGPT-Large-finetuned-chatdoctor

In [ ]:
# https://huggingface.co/Narrativaai/BioGPT-Large-finetuned-chatdoctor
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

model_id = "Narrativaai/BioGPT-Large-finetuned-chatdoctor"
tokenizer = AutoTokenizer.from_pretrained("microsoft/BioGPT-Large")
model = AutoModelForCausalLM.from_pretrained(model_id).to("cuda")

def answer_question(
        prompt,
        temperature=0.1,
        top_p=0.75,
        top_k=40,
        num_beams=2,
        **kwargs,
):
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to("cuda")
    attention_mask = inputs["attention_mask"].to("cuda")
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        **kwargs,
    )
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=512,
            eos_token_id=tokenizer.eos_token_id

        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s, skip_special_tokens=True)
    return output.split(" Response:")[1]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.24M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/566k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/664 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.28G [00:00<?, ?B/s]

In [ ]:
# Get model inference
start = time.time()
answer = answer_question(prompt)
end = time.time()
print(answer)
print(f'Elapsed: {end - start}s')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Hi, Dear thanks for the query to Chat Doctor virtual clinic. I studied your query in full details updated from you. I understood your health concerns. Based on your query data, In my opinion, Dear You seem to suffer from- -Chronic Urticaria with Urticaria Plantar. Treatment-Consult ER Dermatologist who would treat it accordingly. Hope this reply would help you to evaluate your case and treat it with your doctors in time to come. Hope this would resolve your query and worry and Anxiety accompanied by it. Welcome for any further query in this regard to ME. I would love to help you out.Awaiting any further query. Wish you fast recovery from this intriguing health problem. Have a Good Day. Chat Doctor. N.M.S. Genl-CVTS -Senior Surgical Consultant
Elapsed: 24.76878046989441s


## medicine-LLM

In [ ]:
# https://huggingface.co/AdaptLLM/medicine-LLM
from transformers import pipeline

pl = pipeline("text-generation", model="AdaptLLM/medicine-LLM", device_map = 'cuda')

In [ ]:
# Get model inference
start = time.time()
answer = pl(prompt, max_length=512)
end = time.time()
print(answer)
print(f'Elapsed: {end - start}s')

## medalpaca-7b

In [ ]:
# https://huggingface.co/medalpaca/medalpaca-7b
from transformers import pipeline

pl = pipeline("text-generation", model="medalpaca/medalpaca-7b", tokenizer="medalpaca/medalpaca-7b", device_map = 'cuda')

In [ ]:
# Get model inference
start = time.time()
answer = pl(prompt, max_length=512)
end = time.time()
print(answer)
print(f'Elapsed: {end - start}s')

# Server

In [ ]:
!pip install -q starlette uvicorn auto-gptq optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.5/402.5 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not 

In [ ]:
%%bash
cat > server.py <<EOL
from starlette.applications import Starlette
from starlette.responses import JSONResponse
from starlette.routing import Route
import sys
import asyncio
import torch
import tensorflow as tf
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import logging

logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)
tokenizer = None
model = None
prompt_fmt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{0}

### Response:
"""


def drsamantha_init() -> None:
    global tokenizer, model
    tokenizer = AutoTokenizer.from_pretrained("TheBloke/Dr_Samantha-7B-GPTQ", device_map = 'cuda')
    model = AutoModelForCausalLM.from_pretrained("TheBloke/Dr_Samantha-7B-GPTQ", device_map = 'cuda')


def drsamantha_infer(
        prompt: str,
        temperature: float = 0.1,
        top_p: float = 0.75,
        top_k: int = 40,
        num_beams: int = 2,
        **kwargs,
) -> str:
    global tokenizer, model
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to("cuda")
    attention_mask = inputs["attention_mask"].to("cuda")
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        **kwargs,
    )
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=512,
            eos_token_id=tokenizer.eos_token_id

        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s, skip_special_tokens=True)
    torch.cuda.empty_cache()
    return output.split(" Response:")[1]


async def get_model_inference(request) -> JSONResponse:
    global prompt_fmt
    payload = await request.body()
    string = payload.decode("utf-8")
    logging.debug(f"Received POST: {string}")
    response_q = asyncio.Queue()
    await request.app.model_queue.put((prompt_fmt.format(string), response_q))
    output = await response_q.get()
    return JSONResponse(output)


async def server_loop(q) -> None:
    while True:
        (string, response_q) = await q.get()
        logging.debug(f"Calling model inference on: {string}")
        out = drsamantha_infer(string)
        logging.debug(f"Model output: {out}")
        await response_q.put(out)


app = Starlette(
    routes=[
        Route("/api/getModelInf", get_model_inference, methods=["POST"]),
    ],
)


@app.on_event("startup")
async def startup_event() -> None:
    # Assert GPU detected
    assert len(tf.config.list_physical_devices('GPU')) > 0

    # Initialize LLM
    logging.info("Initializing Dr_Samantha model")
    drsamantha_init()

    # Initialize server model queue
    logging.info("Initializing server queue")
    q = asyncio.Queue()
    app.model_queue = q
    asyncio.create_task(server_loop(q))

    # Initialize ngrok tunnelling
    from pyngrok import ngrok

    # Get the dev server port (defaults to 8000 for Uvicorn, can be overridden with `--port`
    # when starting the server
    port = sys.argv[sys.argv.index("--port") + 1] if "--port" in sys.argv else "8000"

    # Open a ngrok tunnel to the dev server
    public_url = ngrok.connect(port).public_url
    logging.info("ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))
EOL

In [ ]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | \
  sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && \
  echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | \
  sudo tee /etc/apt/sources.list.d/ngrok.list && \
  sudo apt update && sudo apt install ngrok -y
!pip install -q pyngrok
!ngrok authtoken 2AEz8HlqT9vSG6MmASHllgaVlc9_QisNKWdZjrU3oWrbapra

deb https://ngrok-agent.s3.amazonaws.com buster main
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://ngrok-agent.s3.amazonaws.com buster InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
32 packages can be upgraded. Run 'apt list --upgradabl

In [ ]:
!uvicorn server:app --port 8000

2024-01-22 14:15:58.636288: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-22 14:15:58.636334: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-22 14:15:58.637539: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-22 14:15:59.807432: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
INFO:     Started server process [10259]
INFO:     Waiting for application startup.
INFO:Initializing Dr_Samantha model
DEBUG:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:https://huggingface.co:443 "HEAD /TheBloke/Dr_Samantha-

In [ ]:
!curl -X POST -d "What is diabetes?" http://localhost:8000/